# PLACE ALL IMPORTS HERE TO AVOID RE-IMPORTING 

In [223]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from pprint import pprint
%pylab inline
import pandas as pd
import os
from collections import Counter
from itertools import chain
import json
import seaborn as sns
import matplotlib.pyplot as plt
import ast

import gc
print(os.getcwd())

Populating the interactive namespace from numpy and matplotlib
/home/cts/Desktop/ML4QS/Assignments/Assignment1_code


# HELPER FUNCTIONS . HERE PUT ALL THE FUNCTIONS ACTUALLY

In [232]:

def get_sensor_values(fname):
    '''Get sensor values function. Parses the .txt log and puts all sensors in a csv'''
    '''
        args:
            fname the filename of the sensor log
        
        output:
            sensor_dict a dictionary where the keys are the sensors and the values are label-vector-timestamp
    '''
    sensors_dict = {}
    with open(fname,'r+') as f:
        for line in f.readlines():
            splitted = line.split('|')
            if len(splitted) != 4:
                #in case we are before our data to avoid parsing errors
                continue
            splitted = [x.strip().lower() for x in splitted]
            label,sensorname,vector,timestamp = splitted
            
            ### same as above this is actually the header  list.
            if sensorname == 'sensorname':
                continue
            label = int(label)
            vector = ast.literal_eval(vector)
            timestamp = int(timestamp)
            if sensorname not in sensor_dict.keys():
                sensor_dict[sensorname] = []
            else:
                tmp_list = sensor_dict.get(sensorname)
                tmp_list.append([label,vector,timestamp])
                sensors_dict[sensorname] = tmp_list
            
    return sensors_dict

def generate_csv(values,header=['label','x','y','z','timestamp'],fname='./data/toy_data/csv/gyroscope.csv'):
    '''Creates a .csv at a specified path for a certain input'''
    '''
        args:
            values    the sensor data values , a list of lists in our case
            header    the header of the .csv . Default is label,x,y,z,timestamp 
            fname     the path and filename to save the .csv file
        
            
    '''
    with open(fname,'w') as f:
        f.write(''.join([x+',' for x in header])+'\n')
        for v in values:
            v[0] = int(v[0])
            v[2] = int(v[2])
            x = str(v[1]).replace('[','').replace(']','').replace(' ','')
            line = '%d,%s,%d\n'%(v[0],x,v[2])
            f.write(line)

In [225]:
#Directory tree should be :
# |-WORKSPACE
# |--thisnotebok.ipynb
# |--datasets
# |--toy_data
# |---sensorLog_01.txt
# |---csv
# |----sensorname.csv
# |----anothersensorname.csv
dataset_folder = './datasets'
current_dataset = 'toy_data'
dataset_fname = 'sensorLog_01.txt'

dataset_path = os.path.join(dataset_folder+'/'+current_dataset,dataset_fname)
csv_path = os.path.join(dataset_folder+'/'+current_dataset,'csv/')
if not os.path.exists(csv_path):
    os.mkdir(csv_path)
cnt = 0
print(csv_path)


./datasets/toy_data/csv/


# Try not to run this cell multiple times when we get more data. This is the file parsing and bottleneck of execution

In [229]:
sensor_dict = get_sensor_values(dataset_path)


In [230]:
#Get sensor names to iterate over them
sensor_names = sensor_dict.keys()
print('==============================================================')
print('Sensors are:',json.dumps(sensor_names,indent=2))

Sensors are: [
  "bmi160 accelerometer", 
  "bmp280 pressure", 
  "rpr0521 light sensor", 
  "gravity", 
  "bmi160 gyroscope", 
  "rotation vector", 
  "linear acceleration"
]


In [233]:
for name in sensor_names:
    fname = name.replace(' ','_')+'.csv'
    generate_csv(sensor_dict[name],fname=csv_path+fname)